Tạo alias cho tên người

In [ ]:
import requests
from bs4 import BeautifulSoup
from unidecode import unidecode

WIKI_URL = "https://en.wikipedia.org/wiki/List_of_Vietnamese_people"

def fetch_names():
    resp = requests.get(WIKI_URL)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    content = soup.find("div", id="mw-content-text")
    names = []
    for li in content.find_all("li"):
        if li.find_previous_sibling(lambda tag: tag.name=="h2" and "See also" in tag.text):
            break
        a = li.find("a")
        if a and a.get("href", "").startswith("/wiki/"):
            name = a.text.strip()
            if name and not name.startswith("List"):
                names.append(name)
    return list(dict.fromkeys(names))

def generate_aliases(full_name):
    parts = full_name.split()
    no_accents = unidecode(full_name)
    initials = ".".join(p[0] for p in parts) + "."
    family = parts[0]
    given = parts[-1]
    fam_init = parts[0][0] + "."
    giv_init = parts[-1][0] + "."

    aliases = {
        full_name,
        no_accents,
        initials,
        family,
        given,
        f"{fam_init} {giv_init}",
        f"{family} {giv_init}",
        f"{fam_init} {given}",
    }
    return [a for a in sorted(aliases) if a and a != full_name]

if __name__ == "__main__":
    names = fetch_names()
    name_dict = {}
    for nm in names:
        name_dict[nm] = generate_aliases(nm)

    import json
    with open("vietnamese_people_aliases.json", "w", encoding="utf-8") as f:
        json.dump(name_dict, f, ensure_ascii=False, indent=2)

    print(f"Đã tạo dict cho {len(name_dict)} tên.")


In [91]:
import os
import json
from unidecode import unidecode

input_path = r"E:\Documents\DS200\final_project\facebook_crawl\list_name_celeb.txt"
output_path = "celebrity_aliases.json"

with open(input_path, "r", encoding="utf-8") as f:
    names = [line.strip() for line in f if line.strip()]


def generate_aliases(name):
    variants = set()
    no_accents = unidecode(name)

    variants.update([
        name,
        name.lower(),
        name.upper(),
        no_accents,
        no_accents.lower(),
        no_accents.upper()
    ])

    if "-" in name:
        parts = name.split("-")
        for part in parts:
            variants.update(generate_aliases(part.strip()))
        variants.add(" ".join(parts))
        variants.add(" ".join(parts).lower())
        variants.add(" ".join(parts).upper())

    if " " in name:
        parts = name.split()
        initials = [p[0] for p in parts if p]

        if len(parts) >= 2:
            # Viết tắt: N.T.T.T
            dot_format = ".".join(initials) + "."
            flat = "".join(initials)
            spaced = " ".join(initials)

            variants.update([
                dot_format, dot_format.lower(), dot_format.upper(),
                flat, flat.lower(), flat.upper(),
                spaced, spaced.lower(), spaced.upper()
            ])

            # Tên đệm + tên
            middle_last = " ".join(parts[-2:])
            variants.update([middle_last, middle_last.lower(), middle_last.upper()])

            # Viết tắt đệm + tên: T.Tiên
            ttien = f"{parts[-2][0]}.{parts[-1]}"
            variants.update([ttien, ttien.lower(), ttien.upper()])

        # In hoa toàn bộ cụm tên
        all_upper = " ".join([p.upper() for p in parts])
        all_lower = " ".join([p.lower() for p in parts])
        variants.update([all_upper, all_lower])

    variants.discard("")
    variants.discard(name)  # giữ bản gốc làm key, không là alias

    return sorted(variants)


alias_dict = {}
for nm in names:
    alias_dict[nm] = generate_aliases(nm)

with open(output_path, 'w', encoding='utf-8') as outf:
    json.dump(alias_dict, outf, ensure_ascii=False, indent=2)

print(f"Generated aliases for {len(alias_dict)} names and saved to {output_path}.")

Generated aliases for 1052 names and saved to celebrity_aliases.json.


Sau khi label bị lỗi đoạn điền label vào file json nên cần đoạn này để xóa phần thừa

In [1]:
import json
import os

# Thư mục chứa các file JSON gốc
folder = r"E:\Documents\DS200\final_project\facebook_crawl\data\json\strict"

# Duyệt qua tất cả 16 file: từ 00 đến 15
for i in range(16):
    original_filename = f"data_labeled_{i:02}.json"
    new_filename = f"data_labeled_{i:02}_cleaned.json"

    original_path = os.path.join(folder, original_filename)
    new_path = os.path.join(folder, new_filename)

    with open(original_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # Cập nhật từng post trong danh sách
    for post in data:
        # Xóa các trường không cần thiết trong comment
        for key in ["Persons", "Aspect_1", "Aspect_2", "Sentiment"]:
            post.get("comment", {}).pop(key, None)

        # Thêm trường Persons mới ở cấp ngoài nếu chưa có
        if "Persons" not in post:
            post["Persons"] = []

    # Ghi ra file mới để so sánh / backup
    with open(new_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

    print(f"✅ Đã tạo: {new_filename}")


✅ Đã tạo: data_labeled_00_cleaned.json
✅ Đã tạo: data_labeled_01_cleaned.json
✅ Đã tạo: data_labeled_02_cleaned.json
✅ Đã tạo: data_labeled_03_cleaned.json
✅ Đã tạo: data_labeled_04_cleaned.json
✅ Đã tạo: data_labeled_05_cleaned.json
✅ Đã tạo: data_labeled_06_cleaned.json
✅ Đã tạo: data_labeled_07_cleaned.json
✅ Đã tạo: data_labeled_08_cleaned.json
✅ Đã tạo: data_labeled_09_cleaned.json
✅ Đã tạo: data_labeled_10_cleaned.json
✅ Đã tạo: data_labeled_11_cleaned.json
✅ Đã tạo: data_labeled_12_cleaned.json
✅ Đã tạo: data_labeled_13_cleaned.json
✅ Đã tạo: data_labeled_14_cleaned.json
✅ Đã tạo: data_labeled_15_cleaned.json


In [2]:
import json
import os

# Thư mục chứa các file cleaned
folder = r"E:\Documents\DS200\final_project\facebook_crawl\data\json\strict"
folder_output = r"E:\Documents\DS200\final_project\facebook_crawl\data\json"
output_file = os.path.join(folder_output, "data_labeled_strict.json")

# Khởi tạo danh sách để chứa toàn bộ dữ liệu
all_data = []

# Duyệt qua tất cả các file cleaned từ 00 đến 15
for i in range(16):
    cleaned_filename = f"data_labeled_{i:02}_cleaned.json"
    cleaned_path = os.path.join(folder, cleaned_filename)

    # Đọc dữ liệu và thêm vào danh sách chung
    with open(cleaned_path, "r", encoding="utf-8") as f:
        part_data = json.load(f)
        all_data.extend(part_data)

    print(f"📥 Đã nạp: {cleaned_filename} ({len(part_data)} mục)")

# Ghi toàn bộ vào file tổng
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(all_data, f, indent=2, ensure_ascii=False)

print(f"\n✅ Đã tạo file tổng: {output_file} ({len(all_data)} mục)")


📥 Đã nạp: data_labeled_00_cleaned.json (374 mục)
📥 Đã nạp: data_labeled_01_cleaned.json (374 mục)
📥 Đã nạp: data_labeled_02_cleaned.json (374 mục)
📥 Đã nạp: data_labeled_03_cleaned.json (374 mục)
📥 Đã nạp: data_labeled_04_cleaned.json (374 mục)
📥 Đã nạp: data_labeled_05_cleaned.json (374 mục)
📥 Đã nạp: data_labeled_06_cleaned.json (374 mục)
📥 Đã nạp: data_labeled_07_cleaned.json (374 mục)
📥 Đã nạp: data_labeled_08_cleaned.json (374 mục)
📥 Đã nạp: data_labeled_09_cleaned.json (374 mục)
📥 Đã nạp: data_labeled_10_cleaned.json (374 mục)
📥 Đã nạp: data_labeled_11_cleaned.json (374 mục)
📥 Đã nạp: data_labeled_12_cleaned.json (374 mục)
📥 Đã nạp: data_labeled_13_cleaned.json (374 mục)
📥 Đã nạp: data_labeled_14_cleaned.json (374 mục)
📥 Đã nạp: data_labeled_15_cleaned.json (362 mục)

✅ Đã tạo file tổng: E:\Documents\DS200\final_project\facebook_crawl\data\json\data_labeled_strict.json (5972 mục)


In [3]:
import json
import os

# Đường dẫn file gốc
folder = r"E:\Documents\DS200\final_project\facebook_crawl\data\json"
input_file = os.path.join(folder, "data_labeled_strict.json")
output_file = os.path.join(folder, "data_labeled_strict_cleaned.json")

def aspect_priority(a):
    if a == "null":
        return 0
    elif a == "Other":
        return 1
    return 2

# Tải dữ liệu
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# Duyệt và chuẩn hóa từng bản ghi
for post in data:
    a1 = str(post.get("Aspect_1", "null") or "null")
    a2 = str(post.get("Aspect_2", "null") or "null")
    sentiments = post.get("Sentiment", ["null", "null"])

    # Đảm bảo sentiment có đủ 2 phần tử
    if len(sentiments) < 2:
        sentiments += ["null"] * (2 - len(sentiments))
    sentiments = [s if s else "null" for s in sentiments]

    # Nếu chỉ có 1 aspect hợp lệ (aspect cụ thể hoặc "Other"), đẩy lên Aspect_1
    if a1 == "null" and a2 != "null":
        a1, a2 = a2, "null"
        sentiments[0], sentiments[1] = sentiments[1], "null"

    elif a1 != "null" and a2 == "null":
        sentiments[1] = "null"

    # Nếu cả hai hợp lệ thì sắp xếp theo ưu tiên
    elif a1 != "null" and a2 != "null":
        if aspect_priority(a2) > aspect_priority(a1):
            a1, a2 = a2, a1
            sentiments[0], sentiments[1] = sentiments[1], sentiments[0]

    # Đảm bảo nếu a2 là "null" thì sentiment[1] cũng là "null"
    if a2 == "null":
        sentiments[1] = "null"

    # Ghi lại vào post
    post["Aspect_1"] = a1
    post["Aspect_2"] = a2
    post["Sentiment"] = sentiments

# Ghi ra file mới
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print(f"✅ Đã chuẩn hóa và lưu vào: {output_file}")


✅ Đã chuẩn hóa và lưu vào: E:\Documents\DS200\final_project\facebook_crawl\data\json\data_labeled_strict_cleaned.json
